In [2]:
# importing relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as px

## Loading

In [3]:
# importing the raw data as a DataFrame
rawdfcsv = pd.read_csv('../data/raw/hospitaldf.csv')

In [4]:
# converting data to parquet file for efficiency
rawdfcsv.to_parquet('rawdata.parquet')

In [47]:
# reading the data
rawdfparquet = pd.read_parquet('../data/raw/rawdata.parquet')

In [14]:
# Reading the dataset to check formatting
rawdfparquet.head()


,Unnamed: 0,dep_name,esi,age,gender,ethnicity,race,lang,religion,maritalstatus,...,cc_vaginaldischarge,cc_vaginalpain,cc_weakness,cc_wheezing,cc_withdrawal-alcohol,cc_woundcheck,cc_woundinfection,cc_woundre-evaluation,cc_wristinjury,cc_wristpain
0,1,B,4.0,40.0,Male,Hispanic or Latino,White or Caucasian,English,None,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,B,4.0,66.0,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,B,2.0,66.0,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,A,2.0,66.0,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,A,3.0,84.0,Female,Hispanic or Latino,Other,Other,Pentecostal,Widowed,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


There seems to be an index column already in the dataset. Since this column matches the index but is shifted by 1, we will drop it. 

In [48]:
# setting the index to be the 'Unnamed: 0' column
rawdfparquet.drop('Unnamed: 0', axis=1, inplace=True)

In [19]:
# checking that the index is now as desired (code commented out)
# awdfparquet.head() 

## Cleaning

### Null values

In [22]:
# checking for null values
null_vals = rawdfparquet.isna().sum().sort_values(ascending=False)

In [24]:
# setting max row display to 1000 so that we can see all columns' null value counts
pd.set_option("display.max_rows", 1000)

In [29]:
# finding percentage of nulls for each collumn 
null_vals_pct = null_vals / rawdfparquet.shape[0] * 100

In [37]:
# having a look at the percentage of nulls in each column

null_vals_pct

phencyclidine(pcp)screen,urine,noconf._median         100.000000
phencyclidine(pcp)screen,urine,noconf._max            100.000000
phencyclidine(pcp)screen,urine,noconf._min            100.000000
phencyclidine(pcp)screen,urine,noconf._last           100.000000
benzodiazepinesscreen,urine,noconf._last               99.999822
epithelialcells_last                                   99.998573
benzodiazepinesscreen,urine,noconf._min                99.997502
benzodiazepinesscreen,urine,noconf._median             99.997502
benzodiazepinesscreen,urine,noconf._max                99.997502
epithelialcells_min                                    99.994469
epithelialcells_median                                 99.994469
epithelialcells_max                                    99.994469
acetonebld_last                                        99.929347
poctroponini._last                                     99.834965
acetonebld_min                                         99.592853
acetonebld_max           

The columns with a significant percentage of null values are mainly medical diagnostics. It would be very time consuming to go through these columns and work out how to deal with them, so we will choose to drop columns that have greater than 25% null values. 

In [45]:
# making a list of column names to drop (columns with more than 25% null values)
need_to_drop = [x for x in null_vals_pct.index if null_vals_pct[x] > 25]

# seeing how many columns we plan on dropping
len(need_to_drop)

387

We are considering dropping 387 columns. This is just over a third of our columns. This seems ok, and leaves us with over 500 columns to choose from as feature variables.

In [49]:
# making a clean dataframe and dropping the columns with more than 25% null values
cleandf = rawdfparquet.drop(need_to_drop, axis=1)

Now that we've dropped columns with over 25% null values, lets see what nulls we have left. 

In [51]:
rem_null_vals = cleandf.isna().sum().sort_values(ascending=False)
rem_null_vals_pct = rem_null_vals / rawdfparquet.shape[0] * 100
rem_null_vals_pct

religion                                              24.121566
arrivalmode                                            3.904469
cc_wristpain                                           0.461207
cc_eyeinjury                                           0.461207
cc_fatigue                                             0.461207
cc_fall>65                                             0.461207
cc_fall                                                0.461207
cc_facialswelling                                      0.461207
cc_facialpain                                          0.461207
cc_faciallaceration                                    0.461207
cc_facialinjury                                        0.461207
cc_eyeredness                                          0.461207
cc_eyeproblem                                          0.461207
cc_eyepain                                             0.461207
cc_extremitylaceration                                 0.461207
cc_extremityweakness                    

'religion' seems to be the most significant column, with just over 24% null values. Let's take a look at the religion column.

In [55]:
# seeing which values 'religion' can take
cleandf['religion'].value_counts()

religion
Catholic             188979
Christian             75326
Baptist               40522
Other                 28954
Pentecostal           25586
Protestant            23346
Jewish                12198
Muslim                 8408
Episcopal              8021
Methodist              4819
Unknown                4649
Jehovah's Witness      4480
Name: count, dtype: int64

There is an 'unknown' column, so we can just fill in unknown values with 'unknown'. Let's do it. 

In [60]:
# filling null values in 'religion' with 'Unknown'
cleandf['religion'] = cleandf['religion'].fillna('Unknown')

The column with the next highest proportion of null values is the 'arrivalmode' column, with just under 4% nulls. Let's have a look at it.

In [66]:
# looking at percentages of each value in the column
cleandf['arrivalmode'].value_counts() / cleandf.shape[0] * 100

arrivalmode
Car                      39.802243
ambulance                33.965166
Walk-in                  20.291497
Other                     1.233572
Public Transportation     0.503848
Wheelchair                0.281541
Police                    0.017663
Name: count, dtype: float64

It may be a good idea to fill the null values with 'Other', though this might be slightly misleading in our analysis. Another option would be to drop the rows with nulls in the 'arrivalmode' column, but we would like to avoid this since we may end up getting rid of a lot of rows if we do this for too many columns with null values. 

'Other' makes up 1.2% of the values in the 'arrivalmode' column. Putting an 'Unknown' option into the column could be the best option, and would make it the 4th most common option in the column. Let's go with this option. 

In [69]:
# filling the nulls with 'Unknown'
cleandf['arrivalmode'] = cleandf['arrivalmode'].fillna('Unknown')

Now lets look at the columns with 0.461207% nulls. There are a lot with this percentage, and it may be that they are all missing values for the same rows. In which case, we can drop the rows with nulls in these columns. 

In [77]:
# updating the remaining null values percentage list
rem_null_vals_pct = rem_null_vals_pct.drop(['religion', 'arrivalmode'], axis=0, errors='ignore')

# fining how many columns have 0.461207% nulls
len([x for x in rem_null_vals_pct.index if rem_null_vals_pct[x] > 0.46120])

200

There are 200 columns with 0.461207% nulls. It seems very unlikely that they are not missing values for the same rows. Let's check this by dropping rows with nulls in the 'cc_wristpain' column. Hopefully this will get red of nulls in all 200 columns above.

In [82]:
# checking the 100 columns all have nulls in the same rows
cleandf.dropna(subset='cc_wristpain').isna().sum().sort_values(ascending=False) / cleandf.shape[0] * 100

esi                                                   0.436764
race                                                  0.004639
age                                                   0.001963
dep_name                                              0.000000
cc_alcoholproblem                                     0.000000
cc_abdominalpain                                      0.000000
cc_abdominalpainpregnant                              0.000000
cc_abnormallab                                        0.000000
cc_abscess                                            0.000000
cc_addictionproblem                                   0.000000
cc_agitation                                          0.000000
cc_alcoholintoxication                                0.000000
cc_alteredmentalstatus                                0.000000
cc_allergicreaction                                   0.000000
cc_abdominalcramping                                  0.000000
cc_animalbite                                         0

This has worked as expected, so we will drop these rows in the dataframe

In [83]:
# dropping the rows
cleandf.dropna(subset='cc_wristpain', inplace=True)

The next column to consider is the 'esi' column, which is our target variable. Since this is our target variable, and such a small percentage (0.44%) are missing, we will drop rows with nulls in this column.

In [87]:
# dropping rows with nulls in the 'esi' column
cleandf.dropna(subset='esi', inplace=True)

Let's check to see if this has also got rid of nulls in the 'race' and 'age' columns

In [89]:
# looking at percentage of nulls in remaining columns
cleandf.isna().sum().sort_values(ascending=False) / cleandf.shape[0] * 100

race                                                  0.004681
age                                                   0.001980
dep_name                                              0.000000
cc_alcoholproblem                                     0.000000
cc_abdominalpain                                      0.000000
cc_abdominalpainpregnant                              0.000000
cc_abnormallab                                        0.000000
cc_abscess                                            0.000000
cc_addictionproblem                                   0.000000
cc_agitation                                          0.000000
cc_alcoholintoxication                                0.000000
cc_allergicreaction                                   0.000000
cc_abdominalcramping                                  0.000000
cc_alteredmentalstatus                                0.000000
cc_animalbite                                         0.000000
cc_ankleinjury                                        0

We still have nulls in the 'race' and 'age' columns, but a very small percentage (less than 0.005% for both). It would make sense just to drop the rows with nulls in these columns, but lets have a quick look at them to see what values they can take.

In [90]:
# looking at the 'race' column
cleandf['race'].value_counts()

race
White or Caucasian                           296801
Black or African American                    156596
Other                                         88615
Asian                                          5683
Patient Refused                                5161
Unknown                                        1688
American Indian or Alaska Native                510
Native Hawaiian or Other Pacific Islander       373
Name: count, dtype: int64

There is an 'Unknown' option, so we will impute the missing values with 'Unknown'

In [92]:
# imputing the missing 'race' values with 'Unknown'
cleandf['race'] = cleandf['race'].fillna('Unknown')

Since we have less than 0.002% null values in the 'age' column, lets just drop these rows.

In [94]:
cleandf.dropna(subset='age', inplace=True)

In [97]:
# checking that there are no more null values in the dataframe
cleandf.isna().sum().sum()

0

There are no more null vallues in the dataframe, and we are ready to moce onto looking into duplicates. First we will save the current dataframe so as no to lose our progress. 

In [98]:
cleandf.to_parquet('../data/cleaned/dataNoNulls.parquet')

### Duplicates

First we will check for column duplicates.

In [103]:
# looking for percentage of duplicate rows.
cleandf.duplicated().sum() / cleandf.shape[0] * 100

0.004500919988045557

0.0045% of the rows are duplicated. Since these are duplicates for over 500 columns, it is safe to assume that the duplicate rows are data for the same patients, so we will drop them.

In [105]:
# dropping duplicates
cleandf.drop_duplicates(inplace=True)

Now let's check for duplicate columns. We will do this by checking for duplicaates on the list of column names, since it would take a long time to check for duplicate rows on the transpose of the dataframe. 

In [110]:
# checking for duplicate columns
cleandf.columns.duplicated().any()

False

There are no duplicate columns. Now our dataframe is free from duplicates and null values. Let's reset the index and save the current dataframe as a parquet file and group the columns.

In [116]:
cleandf.reset_index(drop=True, inplace=True)

In [117]:
# saving the current dataframe as a parquet file
cleandf.to_parquet('../data/cleaned/dataClean.parquet')

## Grouping the columns

In [122]:
# checking the columns that we have
list(cleandf.columns)

['dep_name',
 'esi',
 'age',
 'gender',
 'ethnicity',
 'race',
 'lang',
 'religion',
 'maritalstatus',
 'employstatus',
 'insurance_status',
 'disposition',
 'arrivalmode',
 'arrivalmonth',
 'arrivalday',
 'arrivalhour_bin',
 'previousdispo',
 '2ndarymalig',
 'abdomhernia',
 'abdomnlpain',
 'abortcompl',
 'acqfootdef',
 'acrenlfail',
 'acutecvd',
 'acutemi',
 'acutphanm',
 'adjustmentdisorders',
 'adltrespfl',
 'alcoholrelateddisorders',
 'allergy',
 'amniosdx',
 'analrectal',
 'anemia',
 'aneurysm',
 'anxietydisorders',
 'appendicitis',
 'artembolism',
 'asppneumon',
 'asthma',
 'attentiondeficitconductdisruptivebeha',
 'backproblem',
 'biliarydx',
 'birthasphyx',
 'birthtrauma',
 'bladdercncr',
 'blindness',
 'bnignutneo',
 'bonectcncr',
 'bph',
 'brainnscan',
 'breastcancr',
 'breastdx',
 'brnchlngca',
 'bronchitis',
 'burns',
 'cardiaarrst',
 'cardiacanom',
 'carditis',
 'cataract',
 'cervixcancr',
 'chestpain',
 'chfnonhp',
 'chrkidneydisease',
 'coaghemrdx',
 'coloncancer',
 'com

Our target variable is 'esi'. The project will investigate the effect of various columns (feature variables) on 'esi', and attempt to predict the 'esi' score from inputs for selected feature variables. 

In order to select the feature variables, we will first separate them into groups, and investigate their effect on 'esi' within each group. 

In [11]:
# checking the possible values of the target variable: 'esi'
rawdf['esi'].astype(str).value_counts()


NameError: name 'rawdf' is not defined

First we will select the group 'Demographic', that will comtain all feature variables relating to a patients deographic.

It seems that the first 11 columns, minus the 'esi' column, are related to someones demographic.

In [ ]:
# creating a DataFrame that only includes feature variables relating to a patient's demopgraphic
rawdemdf = rawdfparquet.iloc[:, 0:10]

In [ ]:
# checking to see the demographic dataframe is as desired
rawdemdf.head()

,dep_name,esi,age,gender,ethnicity,race,lang,religion,maritalstatus,employstatus
Unnamed: 0,,,,,,,,,,
1,B,4.0,40.0,Male,Hispanic or Latino,White or Caucasian,English,None,Single,Full Time
2,B,4.0,66.0,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed
3,B,2.0,66.0,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed
4,A,2.0,66.0,Male,Hispanic or Latino,Native Hawaiian or Other Pacific Islander,English,Pentecostal,Married,Not Employed
5,A,3.0,84.0,Female,Hispanic or Latino,Other,Other,Pentecostal,Widowed,Retired


In [ ]:
# saving the demographic dataframe to a parquet file
rawdemdf.to_parquet('rawdemdata.parquet')

The next group we will separate will be the 'cc_' group.

In [ ]:
# creating a new dataframe with the relevant columns
rawccdf = rawdfparquet.loc[:, 'cc_abdominalcramping' : 'cc_wristpain']

In [ ]:
# adding the target variable to the datafram
rawccdf['esi'] = rawdfparquet['esi']

In [ ]:
# saving the cc dataframe to a parquet file
rawccdf.to_parquet('rawccdata.parquet')

We repeat the process with the 'meds_' group.

In [ ]:
# creating the meds dataframe
rawmedsdf = rawdfparquet.loc[:, 'meds_analgesicandantihistaminecombination' : 'meds_vitamins']

In [ ]:
# adding the 'esi' column
rawmedsdf['esi'] = rawdfparquet['esi']

In [ ]:
# saving the meds dataframe to a parquet file
rawmedsdf.to_parquet('../data/raw/rawmedsdata.parquet')

The next group will be 'medical readings'. These will not include the triage or vital readings, which will form separate groups.

In [ ]:
# creating the readings dataframe
rawreadingsdf = rawdfparquet.loc[:, 'absolutelymphocytecount_last' : 'urineculture,routine_count']

In [ ]:
# adding 'esi' column
rawreadingsdf['esi'] = rawdfparquet['esi']

In [ ]:
# saving the readings dataframe to a parquet file
rawreadingsdf.to_parquet('../data/raw/rawreadingsdata.parquet')

The next group will be triage readings.

In [ ]:
# creating the gtriage dataframe
rawtriagedf = rawdfparquet.loc[:, 'triage_vital_hr' : 'triage_vital_temp']

In [ ]:
# adding 'esi' column
rawtriagedf['esi'] = rawdfparquet['esi']

In [ ]:
# saving triage data to a parquet file
rawtriagedf.to_parquet('../data/raw/rawtriagedata.parquet')

The next group will be vitals readings (not including triage readings).

In [ ]:
# creating the vitals dataframe
rawvitalsdf = rawdfparquet.loc[:, 'pulse_last' : 'o2_device_median']

In [ ]:
# adding the 'esi' column
rawvitalsdf['esi'] = rawdfparquet['esi']

In [ ]:
# saving vitals data to parquest file
rawvitalsdf.to_parquet('../data/raw/rawvitalsdata.parquet')

The next group will be previous scans. This will include number of surguries, so as not to leave it in a group of its own. 

In [ ]:
# creating the dataframe
rawscansdf = rawdfparquet.loc[:, 'cxr_count' : 'otherxr_count']

In [ ]:
# adding the 'esi' and 'n_surgeries' columns
rawscansdf['n_surgeries'] = rawdfparquet['n_surgeries']
rawscansdf['esi'] = rawdfparquet['esi']

In [ ]:
# saving the scans data as a parquet file
rawscansdf.to_parquet('../data/raw/rawscansdata.parquet')

The next group will be conditions.

In [ ]:
# creating the dataframe
rawconditionsdf = rawdfparquet.loc[:, '2ndarymalig' : 'whtblooddx']

In [ ]:
# adding 'esi' column
rawconditionsdf['esi'] = rawdfparquet['esi']

In [ ]:
# saving the conditions data as a parquet file
rawconditionsdf.to_parquet('../data/raw/rawconditionsdata.parquet')

The final group we will call info, and will include information such as arrival time, insurance_status, and 'n_edvisits'.

In [ ]:
# creaing the dataframe
rawotherdf = rawdfparquet.loc[:, 'insurance_status' : 'previousdispo']

In [ ]:
# adding 'esi', 'n_edvisits' and 'n_admissions' columns
rawotherdf['esi'] = rawdfparquet['esi']
rawotherdf['n_edvisits'] = rawdfparquet['n_edvisits']
rawotherdf['n_admissions'] = rawdfparquet['n_admissions']

In [ ]:
# saving the other data to a parquet file
rawotherdf.to_parquet('../data/raw/rawotherdata.parquet')

We will check that we have accounted for all columns by summing the columns of each of our dataframes.

In [ ]:
rawotherdf.shape[1] + rawconditionsdf.shape[1] + rawdemdf.shape[1] + rawmedsdf.shape[1] + rawreadingsdf.shape[1] + rawccdf.shape[1] + rawscansdf.shape[1] + rawtriagedf.shape[1] + rawvitalsdf.shape[1]

980

This looks good! we had 972 columns in the inital dataframe, which we have split into 9 dataframes. There is an 'esi' column in all new dataframes, accounting for the extra 8 total columns in all new dataframes. 